In [ ]:
## ECG classification on BIRAFFE2

DATA: 


Timestamp - unix timestamps

Values were recorded with 1 kHz frequency ECG – signal (units: mV)  lowpass filtering in 35 Hz and baseline removal

ECG - the timestamps after series of several connection errors cannot be considered as fully reliable.

Errors are considered to occur in the series if the interval between consecutive error entries in the Procedure log  was less than 0.05 s.


PROCEDURE:

TIMESTAMP – Unix timestamp when the stimuli appeared on the
screen,

ID – subject ID,

COND – one of nine conditions as specified in Sect. 3.3 (P+, P0,
P-, S+, S0, S-, P+S+, P0S0, P-S-),
IADS-ID;IAPS-ID – IADS/IAPS IDs of stimuli. 

Both IADS and IAPS datasets provide Valence/Arousal scores for each stimuli
that can be used for further analyses (these values describe emotions
that were evoked by the stimuli)

ANS-VALENCE;ANS-AROUSAL – values in [1; 9] ranges indicating
the point selected by the subject in the Valence-arousal
faces widget (see Sect. 3.4),

ANS-TIME – response time (0 is a moment when widget appeared
on the screen); NaN indicates that the subject has not made any
choice but left the default option,

EVENT – information about going through the next procedure
checkpoint (e.g. tutorial start, game session end) and about BITalino
errors (see Sect. 4.2).


In [ ]:
import numpy as np
import pandas as pd
import json
import matplotlib
import os
import neurokit2 as nk
pd.set_option('display.max_columns', 200)
from hrvanalysis import remove_outliers, remove_ectopic_beats, interpolate_nan_values
from hrvanalysis import get_time_domain_features, get_geometrical_features, get_frequency_domain_features

In [ ]:
hrv_features =  ['HRV_MeanNN', 'HRV_SDNN', 'HRV_SDANN1', 'HRV_SDNNI1', 'HRV_SDANN2',\
   'HRV_SDNNI2', 'HRV_SDANN5', 'HRV_SDNNI5', 'HRV_RMSSD', 'HRV_SDSD',\
   'HRV_CVNN', 'HRV_CVSD', 'HRV_MedianNN', 'HRV_MadNN', 'HRV_MCVNN',\
   'HRV_IQRNN', 'HRV_pNN50', 'HRV_pNN20', 'HRV_HTI', 'HRV_TINN', 'HRV_ULF',\
   'HRV_VLF', 'HRV_LF', 'HRV_HF', 'HRV_VHF', 'HRV_LFHF', 'HRV_LFn',\
   'HRV_HFn', 'HRV_LnHF', 'HRV_SD1', 'HRV_SD2', 'HRV_SD1SD2', 'HRV_S',\
   'HRV_CSI', 'HRV_CVI', 'HRV_CSI_Modified', 'HRV_PIP', 'HRV_IALS',\
   'HRV_PSS', 'HRV_PAS', 'HRV_GI', 'HRV_SI', 'HRV_AI', 'HRV_PI', 'HRV_C1d',\
   'HRV_C1a', 'HRV_SD1d', 'HRV_SD1a', 'HRV_C2d', 'HRV_C2a', 'HRV_SD2d',\
   'HRV_SD2a', 'HRV_Cd', 'HRV_Ca', 'HRV_SDNNd', 'HRV_SDNNa',\
   'HRV_DFA_alpha1', 'HRV_DFA_alpha1_ExpRange', 'HRV_DFA_alpha1_ExpMean',\
   'HRV_DFA_alpha1_DimRange', 'HRV_DFA_alpha1_DimMean', 'HRV_ApEn',\
   'HRV_SampEn', 'HRV_ShanEn', 'HRV_FuzzyEn', 'HRV_MSE', 'HRV_CMSE',\
   'HRV_RCMSE', 'HRV_CD', 'HRV_HFD', 'HRV_KFD', 'HRV_LZC']
hrv_features_isi = [f'{x}_isi' for x in hrv_features]
hrv_features_diff = [f'{x}_diff' for x in hrv_features]

baseline_before_stimulus = 6
def create_intervals(df):
    #create list with all procedure timestamps:
    timestamp_lst=df['TIMESTAMP'].tolist()
    #create intervals
    intervals = []
    for i in range(len(timestamp_lst)-1):
        current_ = timestamp_lst[i] - baseline_before_stimulus
        next_ = timestamp_lst[i+1]
        intervals.append(next_ - current_)
    last = 20
    intervals.append(last)
    box_list = []
    for i in range(len(timestamp_lst)):
        box_tuple=(f'box_{i}',timestamp_lst[i],timestamp_lst[i] + intervals[i])
        box_list.append(box_tuple)
    return box_list

def box(row, box_list):
    for box in box_list:
        if box[1] <= row and box[2] > row:
            return box[0]
        
def get_overlaping_boxes_df(df_ecg,box_list):
    df_overlapping = pd.DataFrame()
    for box, start, end in box_list:
        df_box = df_ecg.drop(df_ecg[(df_ecg.TIMESTAMP <= start) | (df_ecg.TIMESTAMP >= end)].index)
        df_box.reset_index()
        df_box['BOX'] = df_box.apply(lambda row: box, axis = 1)
        df_overlapping = pd.concat([df_box,df_overlapping], sort = False)
    return df_overlapping.groupby('BOX').ECG.apply(np.array).reset_index()


def get_arousal_valence(row):
    if row['ANS-VALENCE'] > 5 and row['ANS-AROUSAL'] > 5:
        return 3
    elif row['ANS-VALENCE'] > 5 and row['ANS-AROUSAL'] <= 5:
        return 2
    elif row['ANS-VALENCE'] <= 5 and row['ANS-AROUSAL'] > 5:
        return 1 
    elif row['ANS-VALENCE'] <= 5 and row['ANS-AROUSAL'] <= 5:
        return 0

def get_arousal(row):
    if row['ANS-AROUSAL'] > 5:
        return 1
    else:
        return 0
def get_valence(row):
    if row['ANS-VALENCE'] > 5:
        return 1
    else:
        return 0

def get_isi(row):
    chunks =np.array_split(row['ECG'],[6000,18000])
    return np.concatenate((chunks[0],chunks[2]))

def get_reaction(row):
    chunks =np.array_split(row['ECG'],[6000,18000])
    return chunks[1]


def get_hr(array):
    try:
        rpeaks = nk.ecg_findpeaks(array, sampling_rate=fs)['ECG_R_Peaks']
        RR_intervals = np.diff(rpeaks) / fs
        hr = 60 / RR_intervals
        mean_hr = pd.DataFrame(hr).describe().T['mean'][0]
        if mean_hr < 35 or mean_hr > 200:
            return None
        return mean_hr
    except:
        return None


def get_hr_diff(row):
    hr_ici = get_hr(row['ECG_ISI'])
    hr_reaction = get_hr(row['ECG_REACTION'])
    if hr_ici and hr_reaction:
        return hr_reaction/hr_ici

def get_features(array):
    try:
        array = np.array(array)
        peaks,info = nk.ecg_peaks(array, sampling_rate=fs)
        features = nk.hrv(peaks, sampling_rate=fs)
        if features.shape[-1] !=72:
            return [None for x in range(72)]
        else:
            return tuple(features.to_dict('records')[0].values())
    except:
        return [None for x in range(72)]
    
def is_hr_elevated(row):
    if row['HR_DIFF'] > 1:
        return 1
    if row['HR_DIFF'] <= 1:
        return 0
    
def get_features_diff(row):
    result = []
    for i in range(len(hrv_features)):
        try:
            result.append(row[hrv_features[i]] - row[hrv_features_isi[i]])
        except: result.append(None)
    return result
        


In [ ]:
# df['ECG'][0]

# get_features(df['ECG'][0][:10000])

### Create columns with an array of ECG for each stimuli. 12s of reaction recording and 12s of baseline. Baseline is duplicated i.e. 6s is taken from n stimuli, 6s from baseline of n-1. 

In [ ]:
path_procedure = '../BIRAFFE2-procedure'
path_data = '../BIRAFFE2-biosigs'
subjects = os.listdir(path_procedure)
subjects =[x for x in subjects if x.endswith(".csv")]
fs=1000

affective_condition_lst=['P+','P-','S+','S-','PS+','PS-','P0','S0','PS0']

for proc in subjects:
    
    sub_name=proc.split('-')[0]
    print(sub_name)
    try:

        #prepare procedure csv:
        df_procedure = pd.read_csv(f"{path_procedure}/{sub_name}-Procedure.csv",delimiter=';',low_memory = False)
        tutorial_end = df_procedure.loc[df_procedure['EVENT'] == 'TUTORIAL END']['TIMESTAMP'].values[0]
        timestamps_error_occured = df_procedure.loc[df_procedure['EVENT'] == 'BITalino error: The computer lost communication with the device.']['TIMESTAMP'].values
        timestamp_to_cut = None
        for x in timestamps_error_occured:
            if x >= tutorial_end:
                timestamp_to_cut = x
                timestamp_to_cut_index= df_procedure.loc[df_procedure['TIMESTAMP'] == timestamp_to_cut].index[0]

        if timestamp_to_cut:
            df_procedure.drop(df_procedure.index[timestamp_to_cut_index:],inplace=True)
        df_procedure['SUB-NAME'] = df_procedure.apply(lambda row: sub_name, axis=1)
        df_procedure= df_procedure[df_procedure['COND'].isin(affective_condition_lst)]  
        df_procedure=df_procedure.reset_index()
        df_procedure = df_procedure.assign(BOX=pd.Series([f'box_{x}' for x in range(df_procedure.shape[0])]).values)


        #prepare data csv:
        df_ecg = pd.read_csv(f"{path_data}/{sub_name}-BioSigs.csv",low_memory = False)
        del df_ecg['EDA']

        box_list = create_intervals(df_procedure)   
        df_ecg = get_overlaping_boxes_df(df_ecg,box_list)      
        
        #merge:
        df = df_procedure.merge(df_ecg, on='BOX')
        if df.shape[0] == 0: #sometimes timestamps wont align so drop those...
            continue  


        df['AROUSAL_VALENCE_CAT'] = df.apply(lambda row: get_arousal_valence(row), axis=1) 
        df['AROUSAL_CAT'] = df.apply(lambda row: get_arousal(row), axis=1) 
        df['VALENCE_CAT'] = df.apply(lambda row: get_valence(row), axis=1)
        df['ECG_ISI'] = df.apply(lambda row: get_isi(row), axis=1)
        df['ECG_REACTION'] = df.apply(lambda row: get_reaction(row), axis=1)
        df.dropna(subset=['ECG_ISI','ECG_REACTION'], inplace = True)

        df[hrv_features] = df.apply(lambda row: get_features(row['ECG_REACTION']), axis=1,result_type='expand')
        df[hrv_features_isi] = df.apply(lambda row: get_features(row['ECG_ISI']), axis=1,result_type='expand')
        df[hrv_features_diff] = df.apply(lambda row: get_features_diff(row), axis=1,result_type='expand')    


        df['HR_DIFF'] = df.apply(lambda row: get_hr_diff(row), axis=1)
        df['IS_HR_ELEVATED']=df.apply(lambda row: is_hr_elevated(row), axis=1)

        del df['BOX']
        del df['index']
        del df['ECG_REACTION']
        del df['ECG_ISI']
        for x in hrv_features:
            del df[x]
        for x in hrv_features_isi:
            del df[x]
        df.to_csv(f'data-all/data-ecg-preprocessed/ECG_data_{sub_name}.csv',index=False)

        
    except:
        with open('not_processed.txt', 'a') as writer:
            writer.write(f"{sub_name}\n")
#     break
